In [1]:
# imports
import warnings
import numpy as np
import pandas as pd

import torch
import transformers

from datasets import Dataset
from datasets import load_metric

from tqdm import tqdm
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

warnings.filterwarnings("ignore")

C:\Users\ASUS\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Helper functions

In [2]:
def postprocess_text(preds: list, labels: list) -> tuple:
    """Performs post processing on the prediction text and labels"""

    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def prep_data_for_model_fine_tuning(source_lang: list, target_lang: list) -> list:
    """Takes the input data lists and converts into translation list of dicts"""

    data_dict = dict()
    data_dict['translation'] = []

    for sr_text, tr_text in zip(source_lang, target_lang):
        temp_dict = dict()
        temp_dict['ar'] = sr_text
        temp_dict['en'] = tr_text

        data_dict['translation'].append(temp_dict)

    return data_dict

def generate_model_ready_dataset(dataset: list, source: str, target: str,
                                 model_checkpoint: str,
                                 tokenizer: AutoTokenizer):
    """Makes the data training ready for the model"""

    preped_data = []

    for row in dataset:
        inputs = ""+ row[source]
        targets = row[target]

        model_inputs = tokenizer(inputs, max_length=128,
                                 truncation=True, padding=True)

        model_inputs['translation'] = row

        # setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=128,
                                 truncation=True, padding=True)
            model_inputs['labels'] = labels['input_ids']

        preped_data.append(model_inputs)

    return preped_data

def compute_metrics(eval_preds: tuple) -> dict:
    """computes bleu score and other performance metrics """

    metric = load_metric("sacrebleu")
    tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")

    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {'bleu': result['score']}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]

    result['gen_len'] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    return result


### Loading the dataset

In [3]:
data = pd.read_excel('cleaned_data(Arzen).xlsx')
data.head(10)

,egyption_Text,english_Text,category,sub_category
0,الأمير الصغير,The little prince,Novels,the-little-prince
1,أنطوان دي سانت إكزوبيري,Antoine De Saint-Exuper,Novels,the-little-prince
2,الفصل الأول,Chapter 1,Novels,the-little-prince
3,في مره، لما كان عندي ست سنين، شفت صوره روعه، ف...,Once when I was six years old I saw a magnific...,Novels,the-little-prince
4,وكانت عباره عن تعبان من نوع البوا بيبلع فهد.,It was a picture of a boa constrictor in the a...,Novels,the-little-prince
5,ودي نسخة من الرسمه.,Here is a copy of the drawing.,Novels,the-little-prince
6,كان بيقولو في الكتاب: تعابين البوا بيبلعو فريس...,In the book it said: Boa constrictors swallow ...,Novels,the-little-prince
7,ومش بيبقو قادرين يتحركو بعد كدا، وبينامو لمده ...,"After that they are not able to move, and they...",Novels,the-little-prince
8,بعدها، فكرت كثير في مغامرات الأدغال,"I pondered deeply, then, over the adventures o...",Novels,the-little-prince
9,وأنا كمان نجحت في إني أرسم أول رسمه بالألوان ا...,And after some work with a coloured pencil I s...,Novels,the-little-prince


### Split inputs and outputs and perform train/test split

In [4]:
# split inputs and outputs into different sets
inputs = data['egyption_Text']
outputs = data['english_Text']

# apply the train, test, validation split
x_train, x_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.2, random_state = 40)

In [5]:
print('training set: \nx: ',x_train.shape,' y: ',y_train.shape, '\ntest set: \nx: ',x_test.shape,' y: ',y_test.shape)

training set: 
x:  (19084,)  y:  (19084,) 
test set: 
x:  (4772,)  y:  (4772,)


Splitting the training set into **training** and **validation** sets

In [6]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.5, random_state=40)

In [7]:
print('training set: \nx: ',x_train.shape,' y: ',y_train.shape, '\ntest set: \nx: ',x_val.shape,' y: ',y_val.shape)

training set: 
x:  (9542,)  y:  (9542,) 
test set: 
x:  (9542,)  y:  (9542,)


### Tokenize and prep the data using the helper functions

In [8]:
# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")

In [9]:
training_data = prep_data_for_model_fine_tuning(x_train.values, y_train.values)

validation_data = prep_data_for_model_fine_tuning(x_val.values, y_val.values)

test_data = prep_data_for_model_fine_tuning(x_test.values, y_test.values)

In [10]:
# Tokenize the data
train_data = generate_model_ready_dataset(dataset=training_data['translation'],
                                          tokenizer=tokenizer,
                                          source='ar',
                                          target='en',
                                          model_checkpoint="Helsinki-NLP/opus-mt-ar-en")

validation_data = generate_model_ready_dataset(dataset=validation_data['translation'],
                                          tokenizer=tokenizer,
                                          source='ar',
                                          target='en',
                                          model_checkpoint="Helsinki-NLP/opus-mt-ar-en")

test_data = generate_model_ready_dataset(dataset=test_data['translation'],
                                          tokenizer=tokenizer,
                                          source='ar',
                                          target='en',
                                          model_checkpoint="Helsinki-NLP/opus-mt-ar-en")

Converting the list of dictionaries into a pandas dataframe to later on be able to convert it to a dataset object

In [11]:
train_df = pd.DataFrame.from_records(train_data)
validation_df = pd.DataFrame.from_records(validation_data)
test_df = pd.DataFrame.from_records(test_data)

In [12]:
# Convert the dataframes to dataset objects
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
validation_dataset = Dataset.from_pandas(validation_df)

### Loading the model, defining the training arguments and the data collator

In [13]:
# load the model
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ar-en")

**Training Arguments:**
* learning rate: 2e-5
* batch size: 16
* number of epochs: 3

In [14]:
training_args = Seq2SeqTrainingArguments(
    output_dir = './results',  # output directory to store results
    num_train_epochs = 3,
    per_device_train_batch_size = 16, 
    per_device_eval_batch_size = 16, 
    learning_rate = 0.00002, 
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    predict_with_generate=True
)

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

### Training the model with the defined training arguments

In [16]:
trainer = Seq2SeqTrainer(model= model, 
                  args = training_args, 
                  train_dataset=train_dataset, 
                  eval_dataset = validation_dataset,
                  data_collator=data_collator,
                  tokenizer= tokenizer,
                  compute_metrics= compute_metrics)
trainer.train()

  1%|          | 10/1791 [00:28<1:18:46,  2.65s/it]

{'loss': 2.7926, 'grad_norm': 9.328065872192383, 'learning_rate': 1.9888330541596874e-05, 'epoch': 0.02}


  1%|          | 20/1791 [00:58<1:18:50,  2.67s/it]

{'loss': 2.7167, 'grad_norm': 8.610991477966309, 'learning_rate': 1.977666108319375e-05, 'epoch': 0.03}


  2%|▏         | 30/1791 [01:26<1:21:56,  2.79s/it]

...

100%|█████████▉| 1790/1791 [1:14:52<00:02,  2.76s/it]

{'loss': 1.9577, 'grad_norm': 5.891530513763428, 'learning_rate': 1.1166945840312676e-08, 'epoch': 3.0}


100%|██████████| 1791/1791 [1:14:54<00:00,  2.51s/it]

{'train_runtime': 4494.2908, 'train_samples_per_second': 6.369, 'train_steps_per_second': 0.399, 'train_loss': 2.092758018999776, 'epoch': 3.0}


TrainOutput(global_step=1791, training_loss=2.092758018999776, metrics={'train_runtime': 4494.2908, 'train_samples_per_second': 6.369, 'train_steps_per_second': 0.399, 'train_loss': 2.092758018999776, 'epoch': 3.0})

In [17]:
# Save the model 
trainer.save_model('fineTunedHelsinki(16-2e-5-3)')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62833]], 'forced_eos_token_id': 0}


### Evaluate the model

In [18]:
evaluation = trainer.evaluate(test_dataset)
print(evaluation)

100%|██████████| 299/299 [35:30<00:00,  7.13s/it] 

{'eval_loss': 2.0772271156311035, 'eval_bleu': 14.9882, 'eval_gen_len': 12.6119, 'eval_runtime': 2133.3137, 'eval_samples_per_second': 2.237, 'eval_steps_per_second': 0.14, 'epoch': 3.0}


In [19]:
example1 = 'ازيك ايه الاخبار؟'
example2= "مش فاضي. عندي امتحان بكرة"
example3 = "مسافر يوم السبت الجاي"

In [20]:
from transformers import pipeline
translator = pipeline("translation", model = 'fineTunedHelsinki(16-2e-5-3)')
print(translator(example1))
print(translator(example2))
print(translator(example3))

[{'translation_text': 'How are you doing?'}]
[{'translation_text': "I'm not available. I have a test tomorrow."}]
[{'translation_text': "He's going away on Saturday."}]
